# Groq model

# Setup

In [50]:
# Python version
import sys 
print(sys.version)

3.12.2 | packaged by Anaconda, Inc. | (main, Feb 27 2024, 11:35:17) [Clang 14.0.6 ]


In [51]:
# Environment Variables
from dotenv import load_dotenv
import yaml
import os

# Load env
load_dotenv()

True

In [52]:
# Torch config
from torch import cuda, bfloat16, float16
import torch

# Torch options
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [53]:
# Jupyter extensions
!jupyter nbextension enable --py widgetsnbextension

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: dejavu execute kernel kernelspec migrate nbconvert run
troubleshoot trust

Jupyter command `jupyter-nbextension` not found.


# Parameters

In [54]:
# Load parameters from YAML file
with open('../config.yaml', 'r') as file:
    config = yaml.safe_load(file)

In [55]:
# Use optimum
use_optimum = config["use_optimum"]

# Show
use_optimum

False

# Reference

- https://wandb.ai/tensorgirl/uncategorized/reports/Restaurant-menu-ordering-Chatbot-using-Gemma-Langchain-and-Chroma-DB--Vmlldzo2OTUxNjE3
- https://www.youtube.com/watch?v=6dyz2M_UWLwhttps://www.youtube.com/watch?v=6dyz2M_UWLw
- https://www.youtube.com/watch?v=XctooiH0moI&ab_channel=IBMTechnology
- https://www.youtube.com/watch?v=h5wLuVDr0oc&ab_channel=AssemblyAI
- https://www.youtube.com/watch?v=sBhK-2K9bUc&ab_channel=CodingIsFun
- https://www.gradio.app/guides/creating-a-custom-chatbot-with-blocks

# Directory

In [56]:
# Set directory to file location
from pathlib import Path
notebook_location = Path(os.path.abspath(''))
os.chdir(notebook_location)

# Get the current working directory
current_directory = os.getcwd()
current_directory

'/Users/ignasipascual/GitHub/LawGPT/models'

# Libraries

In [57]:
# Langchain
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain import PromptTemplate

# Pinecone
from langchain_pinecone import PineconeVectorStore  
from pinecone import Pinecone

# Groq
from langchain_groq import ChatGroq

In [58]:
# Warnings
import warnings
warnings.filterwarnings("ignore")

# Device

In [59]:
# Setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

# CUDA information
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    allocated_memory = torch.cuda.memory_allocated(0) / (1024**3)  # Convert bytes to GB
    cached_memory = torch.cuda.memory_reserved(0) / (1024**3)  # Convert bytes to GB
    total_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)  # Convert bytes to GB
    available_memory = total_memory - cached_memory
    print('Allocated:   ', round(allocated_memory, 1), 'GB')
    print('Cached:      ', round(cached_memory, 1), 'GB')
    print('Available:  ', round(available_memory, 1), 'GB')
    print('Total:      ', round(total_memory, 1), 'GB')

Using device: cpu



# Pinecone

In [60]:
# Init pinecone
pinecone = Pinecone(api_key = os.environ.get('PINECONE_API_KEY'))

# Connect
index_name = 'lawgpt-unstructured-db'
index = pinecone.Index(index_name)

# Index stats
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.07139,
 'namespaces': {'': {'vector_count': 7139}},
 'total_vector_count': 7139}

# Embeddings

In [61]:
# Model ID
embed_model_id = config["embedding_model"]

# Embed model
embed_model = HuggingFaceEmbeddings(
    model_name = embed_model_id,
    model_kwargs = {'device': device},
    encode_kwargs = {'device': device, 'batch_size': 32}
) 

# Command

In [62]:
# Get pre-prompt
pre_prompt = config["pre_prompt"]

# Create prompt context
prompt_context = config["prompt_context"]

In [63]:
# Google template
template = f"""
<start_of_turn>user
{pre_prompt}. {prompt_context} A continuación se proporciona el contexto: 
Contexto: {{context}} 
Pregunta: {{query}}
<start_of_turn>model
Respuesta: """

In [64]:
# Prompt Template
prompt = PromptTemplate(
    template = template, 
    input_variables = ["context", "query"]
)

# Vector store

In [65]:
# Text field
text_field = "text"  

# Vector store
vectorstore = PineconeVectorStore(index, embed_model, text_field)  

# Show
vectorstore

# LLM Model

In [66]:
# Initialize Groq Langchain chat object and conversation
groq_chat = ChatGroq(
        groq_api_key = os.environ.get('GROQ_API_KEY'), 
        model_name = config["app_model"],
        temperature = config["app_temperature"]
)

# Show
groq_chat

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x163e6fc20>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x163e6dee0>, temperature=1e-08, groq_api_key=SecretStr('**********'))

In [67]:
# Prepare prompt
prompt = ChatPromptTemplate.from_messages([("system", template), ("human", "{query}")])

# Show
prompt

ChatPromptTemplate(input_variables=['context', 'query'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'query'], template='\n<start_of_turn>user\nEres un asistente experto en derecho y leyes españolas y tu objetivo es proporcionar respuestas exhaustivas y precisas a las preguntas planteadas por tus clientes.\nAsegúrate de basar tus respuestas en el contexto proporcionado, utilizando todas las leyes y normativas relevantes para fundamentar tus argumentos.\nEs crucial que todas las respuestas estén redactadas en español y presentadas de forma clara y coherente.\nConsidera ofrecer ejemplos o casos hipotéticos para ilustrar tus puntos de vista.\n. A continuación, se presenta la información relevante que debes usar para responder a las consultas de los clientes. \nEn caso de no encontrar la respuesta, debes indicarlo de forma explícita.\n A continuación se proporciona el contexto: \nContexto: {context} \nPregunta: {query}\n<start_of_turn>model\nResp

In [68]:
# Create chain
chain = prompt | groq_chat

# Show
chain

ChatPromptTemplate(input_variables=['context', 'query'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'query'], template='\n<start_of_turn>user\nEres un asistente experto en derecho y leyes españolas y tu objetivo es proporcionar respuestas exhaustivas y precisas a las preguntas planteadas por tus clientes.\nAsegúrate de basar tus respuestas en el contexto proporcionado, utilizando todas las leyes y normativas relevantes para fundamentar tus argumentos.\nEs crucial que todas las respuestas estén redactadas en español y presentadas de forma clara y coherente.\nConsidera ofrecer ejemplos o casos hipotéticos para ilustrar tus puntos de vista.\n. A continuación, se presenta la información relevante que debes usar para responder a las consultas de los clientes. \nEn caso de no encontrar la respuesta, debes indicarlo de forma explícita.\n A continuación se proporciona el contexto: \nContexto: {context} \nPregunta: {query}\n<start_of_turn>model\nResp

# Run model

In [69]:
# Query
query = "Explicame el articulo 245 del codigo penal español que hace referencia a usurpaciones"

In [70]:
# Similarity output
similarity_output = vectorstore.similarity_search_with_score(query, k = config['top_k_docs'])

# Assuming similarity_output is the list of Document objects
context_processed = []

for doc, score in similarity_output:
    metadata = doc.metadata
    context_processed.append({
        "date": metadata.get('date', ''),
        "title": metadata.get('title', ''),
        "context": doc.page_content,
        "url": metadata.get('url', ''),
        "score": score
    })

# Show
context_processed[0:5]

[{'date': '1995-11-24',
  'title': 'Ley Orgánica 10/1995, de 23 de noviembre, del Código Penal.',
  'context': '2. La misma pena se aplicará al que hiciere uso, a sabiendas, de la certificación falsa.\nCAPITULO III\nDisposición general\nArtículo 400.\nLa fabricación o tenencia de útiles, materiales, instrumentos, sustancias, máquinas, programas de ordenador o aparatos, específicamente destinados a la comisión de los delitos descritos en los capítulos anteriores, se castigarán con la pena señalada en cada caso para los autores.\nCAPITULO IV\nDe la usurpación del estado civil\nArtículo 401.\nEl que usurpare el estado civil de otro será castigado con la pena de prisión de seis meses a tres años.\nCAPITULO V\nDe la usurpación de funciones públicas y del intrusismo\nArtículo 402.\nEl que ilegítimamente ejerciere actos propios de una autoridad o funcionario público atribuyéndose carácter oficial, será castigado con la pena de prisión de uno a tres años.\nArtículo 403.',
  'url': 'https://www

In [71]:
# Invoke response
response = chain.invoke({"context": str(context_processed[0:5]), "query": query})

# Show
response

AIMessage(content='El artículo 245 del Código Penal español hace referencia a la usurpación y se estructura en dos apartados.\n\nEn el primer apartado se especifica que la pena de prisión de seis meses a dos años se impondrá a aquella persona que, con violencia o intimidación en las personas, ocupe una cosa inmueble o usurpe un derecho real inmobiliario de pertenencia ajena. A esta pena se le añadirá una multa de seis a dieciocho meses, que se fijará teniendo en cuenta la utilidad obtenida y el daño causado.\n\nEn el segundo apartado se indica que la pena de multa de tres a seis meses se impondrá a aquella persona que ocupe, sin autorización debida, un inmueble, vivienda o edificio ajenos que no constituyan morada, o se mantenga en ellos contra la voluntad de su titular.\n\nEs importante mencionar que estas penas se impondrán además de las penas en que incurriere por las violencias ejercidas, en caso de que se cometiere el hecho con violencia o intimidación en las personas.', response_

# Clean

In [72]:
# CUDA information
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    allocated_memory = torch.cuda.memory_allocated(0) / (1024**3)  # Convert bytes to GB
    cached_memory = torch.cuda.memory_reserved(0) / (1024**3)  # Convert bytes to GB
    total_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)  # Convert bytes to GB
    available_memory = total_memory - cached_memory
    print('Allocated:   ', round(allocated_memory, 1), 'GB')
    print('Cached:      ', round(cached_memory, 1), 'GB')
    print('Available:  ', round(available_memory, 1), 'GB')
    print('Total:      ', round(total_memory, 1), 'GB')

In [73]:
# Clean memory
torch.cuda.empty_cache()
gc.collect()

93